In [ ]:
import numpy as np 
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from tqdm import tqdm
from progressbar import ProgressBar
import re
from scipy import stats
from pylab import plot, show
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import textwrap
from tabulate import tabulate

In [ ]:
df = pd.read_csv('/content/Books_seperate.csv')
df.drop(['subtitle','isSecondHand','no_of_pages'],axis=1,inplace=True)
df.head(20)

,publication_date,euro_price,type,category,name,author,publisher,language,weight,salesRank,short_blurb
0,20-Aug-20,9.60,Paperback,history,The Royal Netherlands Navy of World War II,"Ryan Noppen, Paul Wright (illustrator)",Bloomsbury Publishing (UK),English,5.0,25358.0,"In the late 19th and early 20th century, a com..."
1,21-Jan-20,2.90,Paperback,literature,Fascism and Democracy,George Orwell (author),Penguin Books Ltd,English,5.6,2852.0,'Fascism and Democracy' collects five brillian...
2,5-Nov-20,10.77,Paperback,fiction,Interior Chinatown,Charles Yu (author),Europa Editions (UK) Ltd.,English,10.0,839.0,"A deeply personal novel about race, pop cultur..."
3,9-Nov-20,12.92,Paperback,artanddesign,Railroad Depots of Northwest Pennsylvania,Dan West (author),Arcadia Publishing,English,16.0,3965.0,From 1860 until the decline of the railroads n...
4,9-Nov-20,12.92,Paperback,sports,Lost Ski Areas of Tahoe and Donner,Ingrid P. Wicken (author),Arcadia Publishing,English,19.0,3371.0,"At the beginning of the twentieth century, Lak..."
5,13-Jul-20,14.36,Paperback,technology,Colorado's Daring Ivy Baldwin,Dr. Jack Stokes Ballard (author),Arcadia Publishing,English,21.0,NaN,"At the beginning of the twentieth century, aer..."
6,1-Oct-20,12.81,Paperback,travel,"K2, the Savage Mountain","Charles S. Houston (author), Robert H. Bates (...",Lyons Press,English,22.0,NaN,The 1953 American expedition to the second hig...
7,3-Jan-13,2.99,Paperback,literature,Politics and the English Language,George Orwell (author),Penguin Books Ltd,English,24.0,3618.0,"Includes all issues such as political issues, ..."
8,3-May-12,5.64,Paperback,medical,Essential Clinical Skills,Susan Carlisle,Taylor and Francis,English,26.0,6958.0,"This handy, pocket-sized book is a quick-refer..."
9,11-Aug-20,7.56,Hardback,stationeryandgift,Harry Potter,Insight Editions,Insight Editions,English,32.0,8778.0,Go behind-the-scenes of the beloved Harry Pott...


In [ ]:
'''import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

for i in range(df['name'].count()):
  df['short_blurb'][i] = cleanhtml(df['short_blurb'][i])
df.head()'''

"import re\n\ndef cleanhtml(raw_html):\n  cleanr = re.compile('<.*?>')\n  cleantext = re.sub(cleanr, '', raw_html)\n  return cleantext\n\nfor i in range(df['name'].count()):\n  df['short_blurb'][i] = cleanhtml(df['short_blurb'][i])\ndf.head()"

In [ ]:
df['language'].fillna('English', inplace=True)

In [ ]:
df['short_blurb'].head(5)

0    In the late 19th and early 20th century, a com...
1    'Fascism and Democracy' collects five brillian...
2    A deeply personal novel about race, pop cultur...
3    From 1860 until the decline of the railroads n...
4    At the beginning of the twentieth century, Lak...
Name: short_blurb, dtype: object

In [ ]:
#removing stopwords, null values
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
df['short_blurb'] = df['short_blurb'].fillna('')

tfidf_matrix = tfidf.fit_transform(df['short_blurb'])

tfidf_matrix.shape

(1250, 11233)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df.index, index=df['name']).drop_duplicates()

In [ ]:
def get_recommendations(weight, cosine_sim=cosine_sim):
    # Get the index of book
    idx = indices[weight]

    # Get the pairwsie similarity scores of all books
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 7 most similar books
    sim_scores = sim_scores[1:8]
    book_indices = [i[0] for i in sim_scores]

    return df.iloc[book_indices]

In [ ]:
def book_details(weight):
  my_dict = df['weight'].to_dict()
  key_list = list(my_dict.keys())
  val_list = list(my_dict.values())
  position = val_list.index(weight)
  p = df.loc[position,['publication_date', 'euro_price', 'type', 'category', 'name',
       'author', 'publisher', 'language',
       'weight', 'salesRank']]
  
  b = get_recommendations(weight)

  
  print(p)
  python_desc = df._get_value(position,'short_blurb')
  print()
  print("Short Description")
  my_wrap = textwrap.TextWrapper(width = 40)
  wrap_list = my_wrap.wrap(text=python_desc)
  for line in wrap_list:
    print(line)
  print()
  print()
  
  
  print(tabulate(b, headers='keys', tablefmt='psql'))  

In [ ]:
weight=int(input())
book_details(weight)

1056
publication_date                                            30-Aug-18
euro_price                                                      25.59
type                                                        Hardback 
category                                                homeandgarden
name                                                         Spectrum
author              Victoria and Albert Museum (associated with work)
publisher                                       Thames and Hudson Ltd
language                                                      English
weight                                                           1056
salesRank                                                         NaN
Name: 992, dtype: object

Short Description
A visual analysis of the colours used in
furnishing fabrics and wallpapers from
the 15th century to now, providing
inspiration for designers.


+------+--------------------+--------------+-----------+--------------+------------------------------------